In [1]:
!pip install -q transformers datasets peft accelerate torch scikit-learn

In [2]:
# import libraries
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel
from peft import get_peft_model, LoraConfig, TaskType

from sklearn.metrics import accuracy_score,f1_score
from tqdm import tqdm

In [3]:
# device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## Load dataset

In [4]:
ds = load_dataset("McGill-NLP/stereoset", "intersentence")

README.md: 0.00B [00:00, ?B/s]

intersentence/validation-00000-of-00001.(…):   0%|          | 0.00/687k [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/2123 [00:00<?, ? examples/s]

## Data preprocessing

In [5]:
texts = []
labels = []
bias_types = []

for ex in ds["validation"]:
    context = ex["context"]
    bias_type = ex["bias_type"]

    sentences = ex["sentences"]["sentence"]
    gold_labels = ex["sentences"]["gold_label"]

    for sent, label in zip(sentences, gold_labels):
        combined = context + " <sep> " + sent
        texts.append(combined)
        labels.append(label)
        bias_types.append(bias_type)

print("Total samples:", len(texts))

Total samples: 6369


In [6]:
from collections import Counter

print("Label distribution:", Counter(labels))
print("Bias types:", Counter(bias_types))

Label distribution: Counter({0: 2123, 1: 2123, 2: 2123})
Bias types: Counter({'race': 2928, 'profession': 2481, 'gender': 726, 'religion': 234})


## Splitting the dataset into train/val/test

In [9]:
# Split: 70% Train, 15% Val, 15% Test
from sklearn.model_selection import train_test_split

train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    texts, labels, test_size=0.3, random_state=42, stratify=labels
)
val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts, temp_labels, test_size=0.5, random_state=42, stratify=temp_labels
)

print(f"Train size: {len(train_texts)}, Val size: {len(val_texts)}, Test size: {len(test_texts)}")

Train size: 4458, Val size: 955, Test size: 956


## Dataset class & Tokenization

In [10]:
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

class StereoSetDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_len, return_tensors="pt")
        self.labels = torch.tensor(labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = StereoSetDataset(train_texts, train_labels, tokenizer)
val_dataset = StereoSetDataset(val_texts, val_labels, tokenizer)
test_dataset = StereoSetDataset(test_texts, test_labels, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

## Load MiniLM with LoRA Configuration

In [11]:
# Load base model
base_model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

# Configure LoRA
peft_config = LoraConfig(
    task_type=TaskType.FEATURE_EXTRACTION,
    r=8,           # Rank
    lora_alpha=32, # Scaling
    lora_dropout=0.1,
    target_modules=["query", "value"]
)

# Apply LoRA to the model
lora_model = get_peft_model(base_model, peft_config)
lora_model.print_trainable_parameters()

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

trainable params: 73,728 || all params: 22,786,944 || trainable%: 0.3236


In [12]:
# Classifier model
class LoRAMiniLMClassifier(torch.nn.Module):
    def __init__(self, lora_encoder):
        super(LoRAMiniLMClassifier, self).__init__()
        self.encoder = lora_encoder
        self.classifier = torch.nn.Linear(384, 3) # Hidden size 384 for MiniLM

    def forward(self, input_ids, attention_mask):
        outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        # Mean pooling of last hidden states
        embeddings = outputs.last_hidden_state.mean(dim=1)
        return self.classifier(embeddings)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LoRAMiniLMClassifier(lora_model).to(device)

optimizer = AdamW(model.parameters(), lr=2e-4) # Slightly higher LR for adapters
criterion = torch.nn.CrossEntropyLoss()

## Model Training

In [16]:
model.train()
for epoch in range(5):
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1} | LoRA Train Loss: {avg_loss:.4f}")

Epoch 1 | LoRA Train Loss: 0.5109
Epoch 2 | LoRA Train Loss: 0.4870
Epoch 3 | LoRA Train Loss: 0.4576
Epoch 4 | LoRA Train Loss: 0.4384
Epoch 5 | LoRA Train Loss: 0.4217


## Evaluation

In [17]:
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask)
        preds = torch.argmax(outputs, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate Metrics
macro_f1 = f1_score(all_labels, all_preds, average='macro')
accuracy = accuracy_score(all_labels, all_preds)

print(f"\n--- LoRA MiniLM Test Results ---")
print(f"Accuracy: {accuracy:.4f}")
print(f"Macro F1 Score: {macro_f1:.4f}")


--- LoRA MiniLM Test Results ---
Accuracy: 0.7500
Macro F1 Score: 0.7456


## Save and zip the model for download

In [20]:
import os, zipfile

save_dir = "lora_minilm_project"
os.makedirs(save_dir, exist_ok=True)

# Save the adapter and the linear head
model.encoder.save_pretrained(os.path.join(save_dir, "adapter"))
torch.save(model.classifier.state_dict(), os.path.join(save_dir, "classifier.pt"))

# Create Zip
zip_name = "lora_minilm.zip"
with zipfile.ZipFile(zip_name, 'w') as z:
    for root, _, files in os.walk(save_dir):
        for f in files:
            z.write(os.path.join(root, f), os.path.relpath(os.path.join(root, f), save_dir))

from google.colab import files
files.download(zip_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>